In [158]:
import sys
import os
import recordlinkage
import numpy as np
import pandas as pd

share_libs_path = os.path.abspath(os.path.join('../..', 'libs'))
# Add the path to sys.path if it's not already there
if share_libs_path not in sys.path:
    sys.path.append(share_libs_path)

try:
    import minio_helper
except ImportError:
    raise ImportError("minio_reader is not found. Please install it from the shared_code folder.")

Start reading data from minio

In [159]:
bucket_name = "csv-data-files"
purchase_csv_file = "purchases.csv"
purchases_df = minio_helper.read_csv(bucket_name, purchase_csv_file)

# s3fs client object
s3fs_client = minio_helper.minio_client()

# define output spaces for re-structured dataframe
FORMATTED_DATA_OUTPUT_BUCKET = "formatted-data-files"

In [160]:
print(purchases_df.shape)
purchases_df.head()

(71519, 11)


,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode
0,2022-10-01 02:26:08+00:00,32701106,2055156924466332447,NaN,shimano,95.21,64c68405-7002-4ce0-9604-a4c2e1f7384b,NaN,MICHAEL,MASON,RG497ZQ
1,2022-10-01 02:28:32+00:00,9400066,2053013566067311601,NaN,jaguar,164.20,3b7d6741-3c82-4c75-8015-6f54b52612e0,7466.0,NaN,NaN,NaN
2,2022-10-01 02:31:01+00:00,1004238,2053013555631882655,electronics.smartphone,apple,1206.40,38c6d3f7-6c32-4fed-bca6-ef98e1746386,NaN,COLE,WILKINSON,SW75TQ
3,2022-10-01 02:33:31+00:00,11300059,2053013555531219353,electronics.telephone,texet,17.48,3398c966-7846-4186-89be-323daad735b9,NaN,MOHAMMED,RICHARDS,RG150RE
4,2022-10-01 02:40:18+00:00,17300751,2053013553853497655,NaN,versace,77.22,11e3a573-01b9-4794-b513-e7d8a4fcac83,31266.0,NaN,NaN,NaN


Profiling dataset

In [161]:
print(purchases_df.isnull().sum())

event_time              0
product_id              0
category_id             0
category_code       16739
brand                5707
price                   0
session_id              0
customer_id         18448
guest_first_name    53071
guest_surname       53071
guest_postcode      53071
dtype: int64


Do analysis

In [162]:
purchases_df['is_guest'] = purchases_df['customer_id'].isnull()

# Check if there are any invalid records that purchase is guest, but the customer_id is presented
invalid_guest_records = purchases_df[purchases_df['is_guest'] & purchases_df['customer_id'].notnull()]
invalid_non_guest_records = purchases_df[(purchases_df['is_guest'] == False) & purchases_df['customer_id'].isnull()]
if invalid_guest_records.empty & invalid_non_guest_records.empty:
    print(f"No invalid records found for guest purchases")

No invalid records found for guest purchases


Check proportion of guest and non-guest checkouts

In [163]:
purchases_df['is_guest'].value_counts(normalize=True)

is_guest
False    0.742055
True     0.257945
Name: proportion, dtype: float64

In [164]:
# Calculate the actual number of guests
guest_names = ['guest_first_name', 'guest_surname', 'guest_postcode']
guest_unique_count = len(purchases_df[guest_names].drop_duplicates())
print(f'There are {guest_unique_count} unique guests')

# Calculate the actual number of customers
unique_customers_count = len(purchases_df['customer_id'].unique())
print(f'There are {unique_customers_count} unique customers')

actual_total = guest_unique_count + unique_customers_count
print(f'Actual total of customers is: {actual_total} from the {len(purchases_df)} purchases records')

# Calculate the proportion
#NOTE: we need to subtract 1 because the null value is included in the count
guest_proportion = guest_unique_count / (actual_total - 1)
print(f'Guest proportion: {guest_proportion}')

There are 8301 unique guests
There are 24962 unique customers
Actual total of customers is: 33263 from the 71519 purchases records
Guest proportion: 0.2495640671036017


Transformation Stage: Restructuring a dataset to the common structure

In [165]:
# first, extract guest data first
guest_columns = ["guest_first_name", "guest_surname", "guest_postcode", "is_guest"]
guests_df = purchases_df.loc[purchases_df['is_guest'] == True, guest_columns]
guests_df.drop_duplicates()
guests_df.head()

,guest_first_name,guest_surname,guest_postcode,is_guest
0,MICHAEL,MASON,RG497ZQ,True
2,COLE,WILKINSON,SW75TQ,True
3,MOHAMMED,RICHARDS,RG150RE,True
7,KIAN,MILLS,SW332TF,True
13,RUBY,OWEN,PO377YS,True


In [166]:
customer_ids_df = (
    pd.DataFrame(
        purchases_df.loc[purchases_df['customer_id'].notnull(), 'customer_id'].unique().astype(int),
        columns=['customer_id']
    )
)

print(len(customer_ids_df))
customer_ids_df.head()

24961


,customer_id
0,7466
1,31266
2,534142828
3,1035
4,6985


Start to combine customer_ids_df and guests_df into a final sales_customer_df

In [167]:
sales_customer_df = pd.concat([customer_ids_df, guests_df], axis=0, ignore_index=True)
print(len(sales_customer_df))
sales_customer_df.head()

43409


,customer_id,guest_first_name,guest_surname,guest_postcode,is_guest
0,7466.0,NaN,NaN,NaN,NaN
1,31266.0,NaN,NaN,NaN,NaN
2,534142828.0,NaN,NaN,NaN,NaN
3,1035.0,NaN,NaN,NaN,NaN
4,6985.0,NaN,NaN,NaN,NaN


Do data transformation for the union dataframe

In [168]:
# Start rename columns data union dataset for better structure format
column_names = ["customer_id", "first_name", "surname", "postcode", "is_guest"]
sales_customer_df = sales_customer_df.set_axis(column_names, axis=1)

# Fill all empty is_guest blank values to False
sales_customer_df['is_guest'] = sales_customer_df['is_guest'].fillna(False)

# Mark that all records from this dataframe are from the purchase source
sales_customer_df['in_purchase_data'] = True

# Do trim all spaces for strings values (we also apply lower case for names components)
for col_name in ['first_name', 'surname']:
    sales_customer_df[col_name] = sales_customer_df[col_name].str.lower().str.strip()
sales_customer_df['postcode'] = sales_customer_df['postcode'].str.strip()
filtered_df = sales_customer_df[sales_customer_df['first_name'].notnull()]
filtered_df

/var/folders/97/835pt_fd54749hk3wzm_wv2w0000gn/T/ipykernel_88740/2233608144.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sales_customer_df['is_guest'] = sales_customer_df['is_guest'].fillna(False)


,customer_id,first_name,surname,postcode,is_guest,in_purchase_data
24961,NaN,michael,mason,RG497ZQ,True,True
24962,NaN,cole,wilkinson,SW75TQ,True,True
24963,NaN,mohammed,richards,RG150RE,True,True
24964,NaN,kian,mills,SW332TF,True,True
24965,NaN,ruby,owen,PO377YS,True,True
...,...,...,...,...,...,...
43404,NaN,harley,thomas,M625UB,True,True
43405,NaN,harley,thomas,M625UB,True,True
43406,NaN,harley,thomas,M625UB,True,True
43407,NaN,harley,thomas,M625UB,True,True


In [169]:
# Save the generated dataframe into Minio bucket for later analysis
minio_helper.upload_df_to_remote(s3fs_client, FORMATTED_DATA_OUTPUT_BUCKET, sales_customer_df, "sales_customer_df.csv")

True

Start with CRM Customer Data

In [170]:
crm_df = minio_helper.read_csv(bucket_name, "crm_export.csv")
print(crm_df.shape)
crm_df.head()

(7825, 5)


,customer_id,first_name,surname,postcode,age
0,29223,Holly,Rogers,LS475RT,12
1,27826,Daniel,Owen,M902XX,5
2,7432,Eleanor,Russell,HR904ZA,34
3,2569,Paige,Roberts,DE732EP,61
4,9195,Matilda,Young,LS670FU,78


In [171]:
crm_df.isnull().sum()

customer_id    0
first_name     0
surname        0
postcode       0
age            0
dtype: int64

Find duplications

In [172]:
crm_df.groupby('customer_id').size().loc[lambda x: x > 1]
for col in ["first_name", "surname"]:
    crm_df[col] = crm_df[col].str.lower().str.strip()

crm_df["postcode"] = crm_df["postcode"].str.strip()

print(len(crm_df))
# find duplicate records without customer_id column
print(len(crm_df.drop(columns='customer_id').drop_duplicates()))

# The percentage of duplication records is too small with size of the dataset, so we decide to keep all the records from the crm dataset

7825
7419


JOINING DATASETS TO ENHANCE ONE WITH INFORMATION FROM ANOTHER

In [173]:
# Do left merge with 2 datasets (sales_customer_df and crm_df), use sales_customer_df as the left
merged_sales_and_crm_df = sales_customer_df.merge(crm_df, on='customer_id', how='left', suffixes=('_sales', '_crm'))
print(len(merged_sales_and_crm_df))
merged_sales_and_crm_df.head()

43409


,customer_id,first_name_sales,surname_sales,postcode_sales,is_guest,in_purchase_data,first_name_crm,surname_crm,postcode_crm,age
0,7466.0,NaN,NaN,NaN,False,True,NaN,NaN,NaN,NaN
1,31266.0,NaN,NaN,NaN,False,True,harley,palmer,HR250EJ,33.0
2,534142828.0,NaN,NaN,NaN,False,True,NaN,NaN,NaN,NaN
3,1035.0,NaN,NaN,NaN,False,True,NaN,NaN,NaN,NaN
4,6985.0,NaN,NaN,NaN,False,True,NaN,NaN,NaN,NaN


In [174]:
# Find duplicates from the merged df
merged_sales_and_crm_df.isnull().sum()

customer_id         18448
first_name_sales    24961
surname_sales       24961
postcode_sales      24961
is_guest                0
in_purchase_data        0
first_name_crm      36295
surname_crm         36295
postcode_crm        36295
age                 36295
dtype: int64

In [175]:
# Find all customers that are valid in merged dataframe
merged_customers_condition = ((merged_sales_and_crm_df["customer_id"].notnull()) & (
        (merged_sales_and_crm_df["first_name_crm"].notnull()) | (merged_sales_and_crm_df["surname_crm"].notnull())))
# Apply to new column call 'in_crm_data' by this above condition
merged_sales_and_crm_df.loc[merged_customers_condition, 'in_crm_data'] = True
merged_sales_and_crm_df.loc[~merged_customers_condition, 'in_crm_data'] = False
merged_sales_and_crm_df['in_crm_data'].value_counts()

in_crm_data
False    36295
True      7114
Name: count, dtype: int64

In [176]:
# Since we have flag that mark customer is from CRM data or not that will be confident for filtering. Start to drop redundant columns
merged_sales_and_crm_df = (
    merged_sales_and_crm_df
    .drop(columns=['first_name_crm', 'surname_crm', 'postcode_crm'])
    .rename(columns={
        'first_name_sales': 'first_name',
        'surname_sales': 'surname',
        'postcode_sales': 'postcode'})
)
print(merged_sales_and_crm_df.shape)
merged_sales_and_crm_df.head()

(43409, 8)


,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data
0,7466.0,NaN,NaN,NaN,False,True,NaN,False
1,31266.0,NaN,NaN,NaN,False,True,33.0,True
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False
3,1035.0,NaN,NaN,NaN,False,True,NaN,False
4,6985.0,NaN,NaN,NaN,False,True,NaN,False


In [177]:
# Now we need to add more CRM customers that are not present in the sales history
crm_ids_to_add = list(set(crm_df["customer_id"].unique()) - set(merged_sales_and_crm_df["customer_id"].unique()))
print(len(crm_ids_to_add))

print(f"Len before adding: {len(merged_sales_and_crm_df)}")
print("Adding customers:")
merged_sales_and_crm_df = pd.concat([merged_sales_and_crm_df, crm_df[crm_df['customer_id'].isin(crm_ids_to_add)]], axis=0, ignore_index=True)
print(f"Len after adding: {len(merged_sales_and_crm_df)}")

merged_sales_and_crm_df.isnull().sum()

711
Len before adding: 43409
Adding customers:
Len after adding: 44120


customer_id         18448
first_name          24961
surname             24961
postcode            24961
is_guest              711
in_purchase_data      711
age                 36295
in_crm_data           711
dtype: int64

In [178]:
# Now filling missing value
merged_sales_and_crm_df["is_guest"] = merged_sales_and_crm_df["is_guest"].fillna(False)
merged_sales_and_crm_df["in_purchase_data"] = merged_sales_and_crm_df["in_purchase_data"].fillna(False)
merged_sales_and_crm_df["in_crm_data"] = merged_sales_and_crm_df["in_crm_data"].fillna(True)

merged_sales_and_crm_df.isnull().sum()

/var/folders/97/835pt_fd54749hk3wzm_wv2w0000gn/T/ipykernel_88740/942038537.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_sales_and_crm_df["is_guest"] = merged_sales_and_crm_df["is_guest"].fillna(False)
/var/folders/97/835pt_fd54749hk3wzm_wv2w0000gn/T/ipykernel_88740/942038537.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_sales_and_crm_df["in_purchase_data"] = merged_sales_and_crm_df["in_purchase_data"].fillna(False)
/var/folders/97/835pt_fd54749hk3wzm_wv2w0000gn/T/ipykernel_88740/942038537.py:4: FutureWarning: Downcasting object

customer_id         18448
first_name          24961
surname             24961
postcode            24961
is_guest                0
in_purchase_data        0
age                 36295
in_crm_data             0
dtype: int64

Exploring third dataset "customer_database"

In [179]:
customer_database_df = minio_helper.read_csv(bucket_name, "customer_database.csv")
print(customer_database_df.shape)
customer_database_df.head()

(23476, 5)


,customer_id,first_name,surname,postcode,age
0,1641,Rhys,Richards,DE456EZ,45
1,24796,Maisie,Young,SW433XX,16
2,14358,Nathan,King,NW49TU,58
3,15306,Jack,Moore,NW908RR,26
4,24971,Alexander,Roberts,SW500HW,85


In [180]:
# Check missing values
customer_database_df.isnull().sum()

customer_id    0
first_name     0
surname        0
postcode       0
age            0
dtype: int64

In [181]:
# Sanitize string values
for col in ["first_name", "surname"]:
    customer_database_df[col] = customer_database_df[col].str.lower().str.strip()

customer_database_df["postcode"] = customer_database_df["postcode"].str.strip()

In [182]:
# Check duplications by customer_id
customer_database_df.groupby('customer_id').size().loc[lambda x: x > 1]

Series([], dtype: int64)

In [183]:
# Check duplications by combination of the name fields
print(f'Len of customer_database_df: {len(customer_database_df)}')
unique_customers = customer_database_df.drop(columns='customer_id').drop_duplicates()
print(f'Len of unique customers: {len(unique_customers)}')
print(f'Found duplications count: {(len(customer_database_df) - len(unique_customers))}')

Len of customer_database_df: 23476
Len of unique customers: 19889
Found duplications count: 3587


Merge all data sources

In [184]:
all_customers_df = merged_sales_and_crm_df.merge(customer_database_df, how='left', on='customer_id',
                                                 suffixes=("_sales", "_customers"))
print(all_customers_df.shape)
all_customers_df.head()

(44120, 12)


,customer_id,first_name_sales,surname_sales,postcode_sales,is_guest,in_purchase_data,age_sales,in_crm_data,first_name_customers,surname_customers,postcode_customers,age_customers
0,7466.0,NaN,NaN,NaN,False,True,NaN,False,eve,richards,HR90PT,45.0
1,31266.0,NaN,NaN,NaN,False,True,33.0,True,NaN,NaN,NaN,NaN
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False,NaN,NaN,NaN,NaN
3,1035.0,NaN,NaN,NaN,False,True,NaN,False,luca,gibson,DE256NH,30.0
4,6985.0,NaN,NaN,NaN,False,True,NaN,False,mia,rogers,HR662RP,43.0


In [185]:
# Start marking customers comes from customer_database
merged_customers_filter = (
    (all_customers_df['customer_id'].notnull()) & (
        (all_customers_df['first_name_customers'].notnull()) & (all_customers_df['surname_customers'].notnull())
)
)
all_customers_df.loc[merged_customers_filter, 'in_customer_database'] = True
all_customers_df.loc[~merged_customers_filter, 'in_customer_database'] = False
all_customers_df['in_customer_database'].value_counts()

in_customer_database
False    22067
True     22053
Name: count, dtype: int64

In [186]:
# We can now update the original customer details, those with a _sales suffix, with details from the customer database
update_filers = (
    (all_customers_df['in_customer_database']) &
    (all_customers_df['first_name_sales'].isnull()) &
    (all_customers_df['surname_sales'].isnull())
)
all_customers_df.loc[update_filers, ['first_name_sales', 'surname_sales', 'postcode_sales', 'age_sales']] = \
    all_customers_df.loc[
        update_filers, ['first_name_customers', 'surname_customers', 'postcode_customers', 'age_customers']].values

# then drop and rename columns
all_customers_df = (
    all_customers_df.drop(columns=['first_name_customers', 'surname_customers', 'postcode_customers', 'age_customers'])
    .rename(columns={
        'first_name_sales': 'first_name',
        'surname_sales': 'surname',
        'postcode_sales': 'postcode',
        'age_sales': 'age'
    })
)
all_customers_df[(all_customers_df['in_purchase_data'] == False)]

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_database
43409,5096.0,oliver,ellis,EX982LZ,False,False,54.0,True,True
43410,18879.0,george,jenkins,RG150RH,False,False,1.0,True,True
43411,6401.0,dylan,bailey,HR529LL,False,False,12.0,True,True
43412,17938.0,louis,walker,PO819ND,False,False,1.0,True,True
43413,14297.0,christopher,robinson,NW481EN,False,False,74.0,True,True
...,...,...,...,...,...,...,...,...,...
44115,7936.0,lily,chapman,LS238QF,False,False,28.0,True,True
44116,12029.0,lilly,clarke,M750HN,False,False,58.0,True,True
44117,11943.0,imogen,campbell,M685ZG,False,False,77.0,True,True
44118,19600.0,mohammed,jackson,RG546PZ,False,False,44.0,True,True


In [187]:
# We also need to add any customers present in the customer database who do not appear in the purchases table
customer_ids_to_add = list(
    set(customer_database_df['customer_id'].unique()) - set(all_customers_df['customer_id'].unique()))
print(f'Found {len(customer_ids_to_add)} customer_ids to add')

Found 1423 customer_ids to add


In [188]:
all_customers_df = (
    pd.concat([all_customers_df, customer_database_df[customer_database_df['customer_id'].isin(customer_ids_to_add)]],
              axis=0, ignore_index=True)
)
print(all_customers_df.shape)

(45543, 9)


In [189]:
# Then, do update the source flags correctly (fill missing values)
all_customers_df['is_guest'] = all_customers_df['is_guest'].fillna(False)
all_customers_df['in_purchase_data'] = all_customers_df['in_purchase_data'].fillna(False)
all_customers_df['in_crm_data'] = all_customers_df['in_crm_data'].fillna(False)
all_customers_df['in_customer_database'] = all_customers_df['in_customer_database'].fillna(False)
all_customers_df = all_customers_df.drop_duplicates()

/var/folders/97/835pt_fd54749hk3wzm_wv2w0000gn/T/ipykernel_88740/2923314591.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_customers_df['is_guest'] = all_customers_df['is_guest'].fillna(False)
/var/folders/97/835pt_fd54749hk3wzm_wv2w0000gn/T/ipykernel_88740/2923314591.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_customers_df['in_purchase_data'] = all_customers_df['in_purchase_data'].fillna(False)
/var/folders/97/835pt_fd54749hk3wzm_wv2w0000gn/T/ipykernel_88740/2923314591.py:4: FutureWarning: Downcasting object dtype arrays on .fillna,

In [190]:
# Now we have the final datasource
print(all_customers_df.shape)
all_customers_df[(all_customers_df['in_purchase_data'] == False)]

(35395, 9)


,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_database
43409,5096.0,oliver,ellis,EX982LZ,False,False,54.0,True,True
43410,18879.0,george,jenkins,RG150RH,False,False,1.0,True,True
43411,6401.0,dylan,bailey,HR529LL,False,False,12.0,True,True
43412,17938.0,louis,walker,PO819ND,False,False,1.0,True,True
43413,14297.0,christopher,robinson,NW481EN,False,False,74.0,True,True
...,...,...,...,...,...,...,...,...,...
45538,14295.0,erin,morgan,NW481EN,False,False,63.0,False,False
45539,28025.0,aaron,harris,SO265RP,False,False,66.0,False,False
45540,4220.0,grace,mitchell,EX709AR,False,False,20.0,False,False
45541,13086.0,oliver,hall,NW277BU,False,False,58.0,False,False


In [191]:
# Start identifying the customers groups
print(f"Number of total customers: {len(all_customers_df)}")
identified_customers = (
    all_customers_df[(all_customers_df["customer_id"].notnull())
                     & (all_customers_df["in_purchase_data"])
                     & ((all_customers_df["in_crm_data"]) | (all_customers_df["in_customer_database"]))
                     ]
)
print(f"Number of identified customers: {len(identified_customers)}")

guest_customers = all_customers_df[all_customers_df['is_guest']]
print(f"Number of guest customers: {len(guest_customers)}")

customer_ids_not_found = (
    all_customers_df[(all_customers_df["customer_id"].notnull())
                  & (all_customers_df["first_name"].isnull())
                  & (all_customers_df["surname"].isnull())]
)
print(f"Number of customer not found: {len(customer_ids_not_found)}")

customer_data_only = (
    all_customers_df[((all_customers_df["in_crm_data"])
                   | (all_customers_df["in_customer_database"])
                  )
                  & (all_customers_df["in_purchase_data"] == False)]
)
print(f"Number of customers data only: {len(customer_data_only)}")

#assert len(all_customers_df) == len(identified_customers) + len(guest_customers) + len(customer_ids_not_found) + len(customer_data_only)


Number of total customers: 35395
Number of identified customers: 23713
Number of guest customers: 8300
Number of customer not found: 3619
Number of customers data only: 711


Start do deduplication data

In [192]:
# Start identifying the customers groups
print(f"Number of total customers: {len(all_customers_df)}")
all_customers_df = all_customers_df.drop_duplicates()
print(f"Number of total customers after deduplication: {len(all_customers_df)}")
all_customers_df.isnull().sum()

Number of total customers: 35395
Number of total customers after deduplication: 35395


customer_id             8300
first_name              3619
surname                 3619
postcode                3619
is_guest                   0
in_purchase_data           0
age                     9548
in_crm_data                0
in_customer_database       0
dtype: int64

In [193]:
# Filling miss values
all_customers_df['customer_id'].agg(['min', 'max'])

all_guests = all_customers_df[all_customers_df['is_guest']].copy()
new_ids = np.arange(-1, -(len(all_guests) + 1), -1)
print(new_ids)

all_customers_df.loc[all_customers_df['is_guest'], 'customer_id'] = new_ids
all_customers_df.isnull().sum()

[   -1    -2    -3 ... -8298 -8299 -8300]


customer_id                0
first_name              3619
surname                 3619
postcode                3619
is_guest                   0
in_purchase_data           0
age                     9548
in_crm_data                0
in_customer_database       0
dtype: int64

Start deduplicate customers based on names and postcode values

In [194]:
considered_columns = ['first_name', 'surname', 'postcode']
duplicates = all_customers_df[all_customers_df.duplicated(considered_columns, keep=False)]
duplicate_dict = duplicates.groupby(considered_columns)['customer_id'].apply(list).to_dict()
print(duplicate_dict)

{('aaliyah', 'harvey', 'SO760SX'): [22648.0, 27397.0], ('aaliyah', 'morgan', 'M190JH'): [10157.0, 30899.0], ('aaliyah', 'robinson', 'DE533TL'): [1905.0, 31960.0], ('aaron', 'barnes', 'LS906PZ'): [9965.0, 32472.0], ('aaron', 'chapman', 'NW436BR'): [28258.0, 13942.0], ('aaron', 'graham', 'HR559UG'): [32092.0, -1485.0], ('aaron', 'holmes', 'EX306WU'): [-3874.0, 3472.0], ('aaron', 'hughes', 'NW68LX'): [-6747.0, 15059.0], ('aaron', 'hunt', 'EX289SL'): [-4358.0, 3335.0], ('aaron', 'martin', 'PO827EN'): [18041.0, -5174.0], ('aaron', 'mason', 'EX675RX'): [27576.0, -6656.0, 4019.0], ('aaron', 'mitchell', 'M432ZY'): [-2310.0, 10883.0], ('aaron', 'parker', 'PO200HH'): [15956.0, 32744.0], ('aaron', 'patel', 'PO61NQ'): [-2597.0, 17525.0], ('aaron', 'richardson', 'HR377QZ'): [-3941.0, 5899.0], ('aaron', 'smith', 'NW886EJ'): [-4413.0, 15248.0], ('aaron', 'williams', 'LS294PR'): [29536.0, 8172.0, 8171.0], ('aaron', 'wilson', 'EX975XA'): [4912.0, 31305.0], ('abbie', 'bennett', 'RG877SY'): [20655.0, 269

In [195]:
# We can use this dictionary to create a new column, other_customer_ids, where we store this list for accounts that have duplicates.
all_customers_df['other_customer_ids'] = all_customers_df.apply(lambda x: duplicate_dict.get((x['first_name'], x['surname'], x['postcode'])), axis=1)
all_customers_df[all_customers_df['other_customer_ids'].notnull()].head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_database,other_customer_ids
5,26434.0,bailey,richardson,SW988AF,False,True,31.0,True,True,"[26434.0, 27761.0]"
6,28961.0,skye,johnson,M80NA,False,True,54.0,False,True,"[28961.0, 12140.0]"
12,22825.0,alicia,wood,SO879UN,False,True,24.0,False,True,"[22825.0, 28495.0]"
28,2187.0,finlay,kelly,DE654YL,False,True,34.0,False,True,"[2187.0, 27589.0]"
32,22478.0,megan,thomson,SO751EG,False,True,52.0,False,True,"[22478.0, 31631.0]"


In [196]:
# Strictly speaking, the other_customer_ids column should not be self-referential, so we should remove a customer’s own ID from it.

In [197]:
def remove_own_id(row):
    """
    Removes a customer's own ID from the list of other customer IDs.

    This function modifies the list of 'other_customer_ids' in the given row by
    removing the value of 'customer_id' from it. The function then returns the
    updated list.

    :param row: A dictionary containing 'customer_id' and 'other_customer_ids'.
                'customer_id' is the customer's unique identifier, while
                'other_customer_ids' is a list of identifiers of other customers.
    :return: The list of 'other_customer_ids' with the 'customer_id' removed.

    """
    ids = list(row['other_customer_ids'])
    ids.remove(row['customer_id'])
    return ids

# We create a new column call 'duplicate_customer_ids' to save actual duplicated other customer ids
all_customers_df.loc[all_customers_df['other_customer_ids'].notnull(), 'duplicate_customer_ids'] = all_customers_df[all_customers_df['other_customer_ids'].notnull()].apply(remove_own_id, axis=1)

In [198]:
# See the result
all_customers_df[all_customers_df['other_customer_ids'].notnull()].head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_database,other_customer_ids,duplicate_customer_ids
5,26434.0,bailey,richardson,SW988AF,False,True,31.0,True,True,"[26434.0, 27761.0]",[27761.0]
6,28961.0,skye,johnson,M80NA,False,True,54.0,False,True,"[28961.0, 12140.0]",[12140.0]
12,22825.0,alicia,wood,SO879UN,False,True,24.0,False,True,"[22825.0, 28495.0]",[28495.0]
28,2187.0,finlay,kelly,DE654YL,False,True,34.0,False,True,"[2187.0, 27589.0]",[27589.0]
32,22478.0,megan,thomson,SO751EG,False,True,52.0,False,True,"[22478.0, 31631.0]",[31631.0]


In [199]:
# We still have duplicate records, so using cumulative count for marking the "main customer" as value is 1
all_customers_df['rank'] = all_customers_df.groupby(considered_columns).cumcount() + 1
all_customers_df.loc[all_customers_df['rank'] == 1, 'is_main'] = True
all_customers_df['is_main'] = all_customers_df['is_main'].fillna(False)
all_customers_df = all_customers_df.drop(columns=['rank'])
all_customers_df.head()

/var/folders/97/835pt_fd54749hk3wzm_wv2w0000gn/T/ipykernel_88740/318765758.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_customers_df['is_main'] = all_customers_df['is_main'].fillna(False)


,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_database,other_customer_ids,duplicate_customer_ids,is_main
0,7466.0,eve,richards,HR90PT,False,True,45.0,False,True,None,NaN,True
1,31266.0,NaN,NaN,NaN,False,True,33.0,True,False,None,NaN,False
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False,False,None,NaN,False
3,1035.0,luca,gibson,DE256NH,False,True,30.0,False,True,None,NaN,True
4,6985.0,mia,rogers,HR662RP,False,True,43.0,False,True,None,NaN,True


In [200]:
print(f"Total customers: {len(all_customers_df)}")
print(f"Unique customers: {len(all_customers_df[all_customers_df['is_main']])}")

Total customers: 35395
Unique customers: 25910


Use recordlinkage to find duplications

In [201]:
indexer = recordlinkage.Index()
indexer.block('postcode') # Tell the code that two records at the same postcode should be tested for duplication. This assumes there are no typos in the postcode
candidate_links = indexer.index(all_customers_df.set_index('customer_id'))

# Create Compare
compare = recordlinkage.Compare()
compare.string('first_name', 'first_name', label='first_name', method='damerau_levenshtein', threshold=0.85)
compare.string('surname', 'surname', label='surname', method='damerau_levenshtein', threshold=0.85)
compare.exact('postcode', 'postcode', label='postcode')

compare_vectors = compare.compute(candidate_links, all_customers_df.set_index('customer_id'))
print(compare_vectors)

                             first_name  surname  postcode
customer_id_1 customer_id_2                               
17152.0        17149.0              0.0      0.0         1
27571.0        25851.0              0.0      0.0         1
16036.0        16046.0              0.0      0.0         1
32207.0        28961.0              0.0      0.0         1
8749.0         8758.0               0.0      0.0         1
...                                 ...      ...       ...
29494.0       -7283.0               0.0      0.0         1
               21565.0              0.0      0.0         1
               21576.0              0.0      0.0         1
               21580.0              0.0      0.0         1
               27390.0              0.0      0.0         1

[883561 rows x 3 columns]


In [202]:
# filter the matches
matches = compare_vectors[compare_vectors.sum(axis=1) == 3]
matches.head()

,,first_name,surname,postcode
customer_id_1,customer_id_2,,,
32298.0,20168.0,1.0,1.0,1
28478.0,17152.0,1.0,1.0,1
27431.0,2012.0,1.0,1.0,1
27842.0,7619.0,1.0,1.0,1
25860.0,27571.0,1.0,1.0,1


In [203]:
# Merge matches into the original customer dataframe
matched_df = pd.DataFrame(
    data=matches.index.tolist(),
    columns=['customer_id_1', 'customer_id_2']
)

matched = all_customers_df.merge(matched_df, left_on='customer_id', right_on='customer_id_1', how='left', suffixes=('_customers', '_matches'))
matched = matched.merge(matched_df, left_on='customer_id', right_on='customer_id_1', how='left', suffixes=('_customers', '_matches'))

#matched.head()

matched[matched['first_name'] == 'harvey'].head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_database,other_customer_ids,duplicate_customer_ids,is_main,customer_id_1_customers,customer_id_2_customers,customer_id_1_matches,customer_id_2_matches
55,17152.0,harvey,owen,PO493ZR,False,True,41.0,True,True,"[17152.0, 28478.0]",[28478.0],True,NaN,NaN,NaN,NaN
75,13101.0,harvey,richardson,NW277BW,False,True,32.0,True,True,None,NaN,True,NaN,NaN,NaN,NaN
189,18166.0,harvey,shaw,PO827ER,False,True,52.0,False,True,None,NaN,True,NaN,NaN,NaN,NaN
309,28478.0,harvey,owen,PO493ZR,False,True,41.0,False,True,"[17152.0, 28478.0]",[17152.0],False,28478.0,17152.0,28478.0,17152.0
498,28105.0,harvey,gibson,RG369QE,False,True,7.0,False,True,"[28105.0, 19124.0]",[19124.0],True,NaN,NaN,NaN,NaN


In [204]:
def merge_duplicates(group):
    duplicate_list = []
    if np.isnan(group['customer_id_1_matches'].values[0]) == False:
        duplicate_list.extend(group['customer_id_1_matches'].tolist())
    if np.isnan(group['customer_id_2_matches'].values[0]) == False:
        duplicate_list.extend(group['customer_id_2_matches'].tolist())
    if len(duplicate_list) > 0:
       return sorted(list(set([int(x) for x in duplicate_list])))
    return np.nan

linkages = matched.groupby("customer_id").apply(merge_duplicates).reset_index(name="linked_duplicates")
linkages.head()

/var/folders/97/835pt_fd54749hk3wzm_wv2w0000gn/T/ipykernel_88740/2778908774.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  linkages = matched.groupby("customer_id").apply(merge_duplicates).reset_index(name="linked_duplicates")


,customer_id,linked_duplicates
0,-8300.0,NaN
1,-8299.0,NaN
2,-8298.0,NaN
3,-8297.0,NaN
4,-8296.0,NaN


In [205]:
all_customers_df = all_customers_df.merge(linkages, on='customer_id', how='left')
#all_customers_df.head()


all_customers_df["duplicate_customer_ids"] = all_customers_df.loc[all_customers_df["duplicate_customer_ids"].notnull(), "duplicate_customer_ids"].apply(lambda x: sorted(x))



results = (
    all_customers_df[(all_customers_df["duplicate_customer_ids"].notnull())
               & (all_customers_df["linked_duplicates"].notnull())
               & (all_customers_df["duplicate_customer_ids"] != all_customers_df["linked_duplicates"])]
)

In [210]:
results[results["first_name"] == 'scarlett']

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_database,other_customer_ids,duplicate_customer_ids,is_main,linked_duplicates
8919,26994.0,scarlett,gibson,LS867AT,False,True,28.0,False,True,"[9785.0, 26994.0]",[9785.0],False,"[9785, 26994]"
10955,32994.0,scarlett,ellis,NW374JP,False,True,43.0,False,True,"[13702.0, 32994.0]",[13702.0],False,"[13702, 32994]"
11763,29805.0,scarlett,shaw,RG546PZ,False,True,22.0,False,True,"[19572.0, 29805.0]",[19572.0],False,"[19572, 29805]"
12652,26777.0,scarlett,ward,DE237GG,False,True,25.0,False,True,"[756.0, 26777.0]",[756.0],False,"[756, 26777]"
13069,15568.0,scarlett,hill,NW967BY,False,True,50.0,False,True,"[31462.0, 15568.0]",[31462.0],False,"[15568, 31462]"
16868,10686.0,scarlett,walker,M352TY,False,True,50.0,True,True,"[27411.0, 10686.0]",[27411.0],False,"[10686, 27411]"
17829,7215.0,scarlett,campbell,HR802JS,False,True,30.0,False,True,"[31592.0, 7215.0]",[31592.0],False,"[7215, 31592]"
18705,29885.0,scarlett,webb,RG720ZR,False,True,33.0,False,True,"[19809.0, 29885.0]",[19809.0],False,"[19809, 29885]"
19035,9179.0,scarlett,harrison,LS670FU,False,True,5.0,False,True,"[27076.0, 9179.0]",[27076.0],False,"[9179, 27076]"
20889,24409.0,scarlett,barnes,SW332TG,False,True,35.0,False,True,"[30212.0, 24409.0]",[30212.0],False,"[24409, 30212]"
